In [18]:
import qml 

In [19]:
from glob import glob
import numpy as np

In [20]:
from rdkit import Chem

In [21]:
def read_sdf(sdf):
    with open(sdf, "r") as f:
        txt = f.read().rstrip()
    return txt

In [22]:
def get_ncharges_coords(sdf):
    mol = Chem.MolFromMolBlock(sdf)
    # rdkit molobj
    ncharges = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    conf = mol.GetConformer()
    coords = np.asarray(conf.GetPositions())
    return ncharges, coords

In [23]:
target_sdfs = sorted(glob("../targets/*.sdf"))
target_sdfs

['../targets/qm9.sdf', '../targets/vitc.sdf', '../targets/vitd.sdf']

In [24]:
qm9_amons_files = sorted(glob("../amons-qm9/*.sdf"))

In [25]:
qm9_amons_sdfs = [read_sdf(x) for x in qm9_amons_files]

In [26]:
conf_data = [get_ncharges_coords(x) for x in qm9_amons_sdfs]

In [27]:
ncharges_list, coords_list = zip(*conf_data)

In [28]:
qm9_ncharges = ncharges_list

In [29]:
mbtypes = qml.representations.get_slatm_mbtypes(qm9_ncharges)

In [30]:
qm9_reps = [np.array(qml.representations.generate_slatm(coords_list[i], ncharges_list[i], mbtypes,
                                              local=False, rcut=8)) for i in 
       range(len(ncharges_list))]

In [31]:
qm9_reps = np.array(qm9_reps)

In [32]:
qm9_reps[0].shape

(3763,)

In [33]:
qm9_amons_labels = [t.split("/")[-1].split(".sdf")[0] for t in qm9_amons_files]

In [34]:
# np save 

In [35]:
np.savez("../representations/amons_SLATM_8_global_data.npz", 
         qm9_amons_labels=qm9_amons_labels,
         qm9_amons_ncharges=qm9_ncharges,
         qm9_amons_reps=qm9_reps)